## Who Uses Cyclistic: Members vs Casuals

### Business Task:
The goal of this analysis is to explore and compare usage patterns between Cyclistic's casual riders and annual members. The objective is to uncover behavioral trends and insights that can inform targeted marketing strategies. The analysis aims to identify key differences in ride frequency, duration, time of day, and trip purpose between these two rider segments. These insights will serve as the foundation for a strategic initiative to convert casual riders into long-term members, contributing to Cyclistic’s growth and profitability.

### Data Source:

For this analysis, I utilized publicly available historical trip data provided by Motivate International Inc. under a permissive license for educational and analytical purposes. Specifically, I worked with the **Divvy 2019 Q1** and **Divvy 2020 Q1** datasets. These datasets represent anonymized bike-share trip logs for Chicago’s Divvy system, which is used here to simulate Cyclistic’s operations.

The Divvy 2019 Q1 dataset contains structured records including:
- Ride identifiers (trip_id)
- Timestamps (start_time, end_time) in the format yyyy-mm-dd hh:mm:ss
- User details (usertype: subscriber or customer, gender, birthyear)
- Bike identifier (bikeid)
- Origin station details (from_station_id, from_station_name)
- Destination station details (to_station_id, to_station_name)
- Trip duration (tripduration) in seconds

The Divvy 2020 Q1 dataset contains structured records including:
- Ride identifiers (ride_id, rideable_type)
- Timestamps (started_at, ended_at) in the format yyyy-mm-dd hh:mm:ss
- User details (member_casual: member or casual)
- Origin station details (start_station_id, start_station_name, start_lat, start_lng)
- Destination station details (end_station_id, end_station_name, end_lat, end_lng)

All personally identifiable information has been excluded to preserve privacy and security.

### Data Preparation:

Install *tidyverse*, *conflicted*, and *scales* libraries. *tidyverse* is a collection of R packages for data science workflows. It includes *ggplot2* for visualization, *dplyr* for data wrangling, *tidyr* for tidying data, *readr* for reading CSVs, *purrr* for functional programming, and *tibble* for modern data frames. *conflicted*, on the other hand, is a library that helps manage namespace conflicts between R packages. *scales* provides functions and formatting tools for data visualization.

In [2]:
install.packages("tidyverse")
install.packages("conflicted")
install.packages("scales")

Installing package into ‘/home/laballais/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

Installing package into ‘/home/laballais/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

Installing package into ‘/home/laballais/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



Load the libraries and set default choices for library conflicts.

In [3]:
library("tidyverse")
library("conflicted")
library("scales")

conflict_prefer("filter", "dplyr")
conflict_prefer("lag", "dplyr")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
[conflicted] Will prefer dplyr::filter over any other package.
[conflicted] Will prefer dplyr::lag over any other package.


Save the Divvy datasets (csv) into dataframes.

In [15]:
# Datasets are assumed to be located inside a folder named "dataset" which is in the same location as this file. 
q1_2019 <- read_csv("./dataset/Divvy_Trips_2019_Q1.csv")
q1_2020 <- read_csv("./dataset/Divvy_Trips_2020_Q1.csv")

Rows: 365069 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): start_time, end_time, from_station_name, to_station_name, usertype,...
dbl (5): trip_id, bikeid, from_station_id, to_station_id, birthyear
num (1): tripduration

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 426887 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): ride_id, rideable_type, started_at, ended_at, start_station_name, e...
dbl (6): start_station_id, end_station_id, start_lat, start_lng, end_lat, en...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Check the column names of the two dataframes.

In [16]:
colnames(q1_2019)
colnames(q1_2020)

[1] "trip_id"           "start_time"        "end_time"         
 [4] "bikeid"            "tripduration"      "from_station_id"  
 [7] "from_station_name" "to_station_id"     "to_station_name"  
[10] "usertype"          "gender"            "birthyear"

[1] "ride_id"            "rideable_type"      "started_at"        
 [4] "ended_at"           "start_station_name" "start_station_id"  
 [7] "end_station_name"   "end_station_id"     "start_lat"         
[10] "start_lng"          "end_lat"            "end_lng"           
[13] "member_casual"

Follow the column names of q1_2020. Rename the column names of q1_2019 and check the two dataframes for inconsistencies.

In [17]:
q1_2019 <- rename(q1_2019,
                     ride_id = trip_id,
                     rideable_type = bikeid,
                     started_at = start_time,
                     ended_at = end_time,
                     start_station_name = from_station_name,
                     start_station_id = from_station_id,
                     end_station_name = to_station_name,
                     end_station_id = to_station_id,
                     member_casual = usertype
              )

str(q1_2019)
str(q1_2020)

spc_tbl_ [365,069 × 12] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ ride_id           : num [1:365069] 21742443 21742444 21742445 21742446 21742447 ...
 $ started_at        : chr [1:365069] "2019-01-01 0:04:37" "2019-01-01 0:08:13" "2019-01-01 0:13:23" "2019-01-01 0:13:45" ...
 $ ended_at          : chr [1:365069] "2019-01-01 0:11:07" "2019-01-01 0:15:34" "2019-01-01 0:27:12" "2019-01-01 0:43:28" ...
 $ rideable_type     : num [1:365069] 2167 4386 1524 252 1170 ...
 $ tripduration      : num [1:365069] 390 441 829 1783 364 ...
 $ start_station_id  : num [1:365069] 199 44 15 123 173 98 98 211 150 268 ...
 $ start_station_name: chr [1:365069] "Wabash Ave & Grand Ave" "State St & Randolph St" "Racine Ave & 18th St" "California Ave & Milwaukee Ave" ...
 $ end_station_id    : num [1:365069] 84 624 644 176 35 49 49 142 148 141 ...
 $ end_station_name  : chr [1:365069] "Milwaukee Ave & Grand Ave" "Dearborn St & Van Buren St (*)" "Western Ave & Fillmore St (*)" "Clark St & Elm St" ...
 $ member

spc_tbl_ [426,887 × 13] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ ride_id           : chr [1:426887] "EACB19130B0CDA4A" "8FED874C809DC021" "789F3C21E472CA96" "C9A388DAC6ABF313" ...
 $ rideable_type     : chr [1:426887] "docked_bike" "docked_bike" "docked_bike" "docked_bike" ...
 $ started_at        : chr [1:426887] "2020-01-21 20:06:59" "2020-01-30 14:22:39" "2020-01-09 19:29:26" "2020-01-06 16:17:07" ...
 $ ended_at          : chr [1:426887] "2020-01-21 20:14:30" "2020-01-30 14:26:22" "2020-01-09 19:32:17" "2020-01-06 16:25:56" ...
 $ start_station_name: chr [1:426887] "Western Ave & Leland Ave" "Clark St & Montrose Ave" "Broadway & Belmont Ave" "Clark St & Randolph St" ...
 $ start_station_id  : num [1:426887] 239 234 296 51 66 212 96 96 212 38 ...
 $ end_station_name  : chr [1:426887] "Clark St & Leland Ave" "Southport Ave & Irving Park Rd" "Wilton Ave & Belmont Ave" "Fairbanks Ct & Grand Ave" ...
 $ end_station_id    : num [1:426887] 326 318 117 24 212 96 212 212 96 100 ...
 $ sta

*ride_id* and *rideable_type* attributes in the two dataframes are of different datatypes. Convert *ride_id* and *rideable_type* into consistent datatypes for the two dataframes.

In [19]:
q1_2019 <-  mutate(q1_2019, ride_id = as.character(ride_id), rideable_type = as.character(rideable_type))

str(q1_2019)
str(q1_2020)

tibble [365,069 × 12] (S3: tbl_df/tbl/data.frame)
 $ ride_id           : chr [1:365069] "21742443" "21742444" "21742445" "21742446" ...
 $ started_at        : chr [1:365069] "2019-01-01 0:04:37" "2019-01-01 0:08:13" "2019-01-01 0:13:23" "2019-01-01 0:13:45" ...
 $ ended_at          : chr [1:365069] "2019-01-01 0:11:07" "2019-01-01 0:15:34" "2019-01-01 0:27:12" "2019-01-01 0:43:28" ...
 $ rideable_type     : chr [1:365069] "2167" "4386" "1524" "252" ...
 $ tripduration      : num [1:365069] 390 441 829 1783 364 ...
 $ start_station_id  : num [1:365069] 199 44 15 123 173 98 98 211 150 268 ...
 $ start_station_name: chr [1:365069] "Wabash Ave & Grand Ave" "State St & Randolph St" "Racine Ave & 18th St" "California Ave & Milwaukee Ave" ...
 $ end_station_id    : num [1:365069] 84 624 644 176 35 49 49 142 148 141 ...
 $ end_station_name  : chr [1:365069] "Milwaukee Ave & Grand Ave" "Dearborn St & Van Buren St (*)" "Western Ave & Fillmore St (*)" "Clark St & Elm St" ...
 $ member_casual     

Transform the two dataframes into a single dataframe.

In [20]:
all_trips <- bind_rows(q1_2019, q1_2020)

Remove columns for tripduration, lat, long, birthyear, and gender since these data were dropped in Divvy datasets starting 2020.

In [22]:
all_trips <- all_trips %>%
    select(-c(start_lat, start_lng, end_lat, end_lng, birthyear, gender,  "tripduration"))

### Data Cleaning and Processing:

Inspect the new dataframe.

In [36]:
colnames(all_trips)  
nrow(all_trips) 
dim(all_trips)  
head(all_trips) 
str(all_trips)
summary(all_trips)

[1] "ride_id"            "started_at"         "ended_at"          
[4] "rideable_type"      "start_station_id"   "start_station_name"
[7] "end_station_id"     "end_station_name"   "member_casual"

[1] 791956

[1] 791956      9

ride_id,started_at,ended_at,rideable_type,start_station_id,start_station_name,end_station_id,end_station_name,member_casual
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
21742443,2019-01-01 0:04:37,2019-01-01 0:11:07,2167,199,Wabash Ave & Grand Ave,84,Milwaukee Ave & Grand Ave,Subscriber
21742444,2019-01-01 0:08:13,2019-01-01 0:15:34,4386,44,State St & Randolph St,624,Dearborn St & Van Buren St (*),Subscriber
21742445,2019-01-01 0:13:23,2019-01-01 0:27:12,1524,15,Racine Ave & 18th St,644,Western Ave & Fillmore St (*),Subscriber
21742446,2019-01-01 0:13:45,2019-01-01 0:43:28,252,123,California Ave & Milwaukee Ave,176,Clark St & Elm St,Subscriber
21742447,2019-01-01 0:14:52,2019-01-01 0:20:56,1170,173,Mies van der Rohe Way & Chicago Ave,35,Streeter Dr & Grand Ave,Subscriber
21742448,2019-01-01 0:15:33,2019-01-01 0:19:09,2437,98,LaSalle St & Washington St,49,Dearborn St & Monroe St,Subscriber


tibble [791,956 × 9] (S3: tbl_df/tbl/data.frame)
 $ ride_id           : chr [1:791956] "21742443" "21742444" "21742445" "21742446" ...
 $ started_at        : chr [1:791956] "2019-01-01 0:04:37" "2019-01-01 0:08:13" "2019-01-01 0:13:23" "2019-01-01 0:13:45" ...
 $ ended_at          : chr [1:791956] "2019-01-01 0:11:07" "2019-01-01 0:15:34" "2019-01-01 0:27:12" "2019-01-01 0:43:28" ...
 $ rideable_type     : chr [1:791956] "2167" "4386" "1524" "252" ...
 $ start_station_id  : num [1:791956] 199 44 15 123 173 98 98 211 150 268 ...
 $ start_station_name: chr [1:791956] "Wabash Ave & Grand Ave" "State St & Randolph St" "Racine Ave & 18th St" "California Ave & Milwaukee Ave" ...
 $ end_station_id    : num [1:791956] 84 624 644 176 35 49 49 142 148 141 ...
 $ end_station_name  : chr [1:791956] "Milwaukee Ave & Grand Ave" "Dearborn St & Van Buren St (*)" "Western Ave & Fillmore St (*)" "Clark St & Elm St" ...
 $ member_casual     : chr [1:791956] "Subscriber" "Subscriber" "Subscriber" "Subscri

   ride_id           started_at          ended_at         rideable_type     
 Length:791956      Length:791956      Length:791956      Length:791956     
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                                                            
                                                                            
 start_station_id start_station_name end_station_id  end_station_name  
 Min.   :  2.0    Length:791956      Min.   :  2.0   Length:791956     
 1st Qu.: 77.0    Class :character   1st Qu.: 77.0   Class :character  
 Median :174.0    Mode  :character   Median :174.0   Mode  :character  
 Mean   :204.4                       Mean   :204.4                     
 3rd Qu.:291.0          

Recalling the description earlier on the data source, 2019 and 2020 Divvy datasets use different nomenclatures for the users. In the 2019 dataset, users are categorized as either *Subscriber* or *Customer*. In the 2020 dataset, users are categorized as either *member* or *casual*. 

In [37]:
table(all_trips$member_casual)


    casual   Customer     member Subscriber 
     48480      23163     378407     341906 

There are currently four categories under the *member_casual* column. Use a consistent nomenclature for member_casual by following the more recent dataframe (2020). Reassign *Subscriber* as *member* and *Customer* as *casual*.

In [38]:
all_trips <-  all_trips %>% 
  mutate(member_casual = recode(member_casual,
                                "Subscriber" = "member",
                                "Customer" = "casual"))

table(all_trips$member_casual)


casual member 
 71643 720313 

Add information on month, day, year, day of the week for each record.

In [39]:
all_trips$date <- as.Date(all_trips$started_at)
all_trips$month <- format(as.Date(all_trips$date), "%m")
all_trips$day <- format(as.Date(all_trips$date), "%d")
all_trips$year <- format(as.Date(all_trips$date), "%Y")
all_trips$day_of_week <- format(as.Date(all_trips$date), "%A")

Add *ride_length* calculation for each record. Inspect the structure of the dataframe.

In [ ]:
all_trips$ride_length <- difftime(all_trips$ended_at,all_trips$started_at)  # result is in seconds
str(all_trips)

tibble [791,956 × 15] (S3: tbl_df/tbl/data.frame)
 $ ride_id           : chr [1:791956] "21742443" "21742444" "21742445" "21742446" ...
 $ started_at        : chr [1:791956] "2019-01-01 0:04:37" "2019-01-01 0:08:13" "2019-01-01 0:13:23" "2019-01-01 0:13:45" ...
 $ ended_at          : chr [1:791956] "2019-01-01 0:11:07" "2019-01-01 0:15:34" "2019-01-01 0:27:12" "2019-01-01 0:43:28" ...
 $ rideable_type     : chr [1:791956] "2167" "4386" "1524" "252" ...
 $ start_station_id  : num [1:791956] 199 44 15 123 173 98 98 211 150 268 ...
 $ start_station_name: chr [1:791956] "Wabash Ave & Grand Ave" "State St & Randolph St" "Racine Ave & 18th St" "California Ave & Milwaukee Ave" ...
 $ end_station_id    : num [1:791956] 84 624 644 176 35 49 49 142 148 141 ...
 $ end_station_name  : chr [1:791956] "Milwaukee Ave & Grand Ave" "Dearborn St & Van Buren St (*)" "Western Ave & Fillmore St (*)" "Clark St & Elm St" ...
 $ member_casual     : chr [1:791956] "member" "member" "member" "member" ...
 $ dat

Convert *ride_length* to numeric datatype.

In [44]:
all_trips$ride_length <- as.numeric(as.character(all_trips$ride_length))
is.numeric(all_trips$ride_length)

[1] TRUE

When you inspect the *start_station_name* field, one value is *HQ QR*. These must be trips where the bikes were taken out of the docks and serviced by employees and not by customers. These records will be deleted from our data. Similarly, records with *end_station_name* field at *HQ QR* will also be deleted.

Records with negative *ride_length* will also be deleted.

In [51]:
table(all_trips$start_station_name)
table(all_trips$end_station_name)
table(all_trips$ride_length)


                        2112 W Peterson Ave 
                                         87 
                              63rd St Beach 
                                        187 
                          900 W Harrison St 
                                       2046 
                 Aberdeen St & Jackson Blvd 
                                       2877 
                    Aberdeen St & Monroe St 
                                       2596 
                  Aberdeen St & Randolph St 
                                       2131 
                   Ada St & Washington Blvd 
                                       1720 
                          Adler Planetarium 
                                       1642 
                       Albany Ave & 26th St 
                                         36 
              Albany Ave & Bloomingdale Ave 
                                        818 
                  Albany Ave & Montrose Ave 
                                        140 
         


                        2112 W Peterson Ave 
                                        142 
                              63rd St Beach 
                                        174 
                          900 W Harrison St 
                                       1836 
                 Aberdeen St & Jackson Blvd 
                                       2863 
                    Aberdeen St & Monroe St 
                                       2570 
                  Aberdeen St & Randolph St 
                                       2214 
                   Ada St & Washington Blvd 
                                       1770 
                          Adler Planetarium 
                                       1463 
                       Albany Ave & 26th St 
                                         36 
              Albany Ave & Bloomingdale Ave 
                                        807 
                  Albany Ave & Montrose Ave 
                                        220 
         


    -552      -93      -79      -23      -22      -20      -17      -16 
       1        1        1        1        1        1        2        2 
     -15      -14      -13      -12      -11      -10       -4       -2 
       1        7       22       37       33        4        1        1 
      -1        0        1        2        3        4        5        6 
       1       93      270     2046     1106      386      222      210 
       7        8        9       10       11       12       13       14 
     155      150      119      125      100      112       94       92 
      15       16       17       18       19       20       21       22 
      95       83       96       80       76       79       58       73 
      23       24       25       26       27       28       29       30 
      78       71       90       96       73       57       52       50 
      31       32       33       34       35       36       37       38 
      47       35       31       52       44      

Delete the records with bad data: *start_station_name* and *end_station_name* at *HQ QR* and negative *ride_length*.

In [61]:
all_trips_cleaned <- all_trips[!(all_trips$start_station_name == "HQ QR" | all_trips$ride_length<0),]

Summarize analysis metrics on *ride_length*.

In [62]:
summary(all_trips_cleaned$ride_length)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
       1      331      539     1189      912 10632022 

Compare metrics between member and casual cyclists.

In [63]:
aggregate(all_trips_cleaned$ride_length ~ all_trips_cleaned$member_casual, FUN = mean)
aggregate(all_trips_cleaned$ride_length ~ all_trips_cleaned$member_casual, FUN = median)
aggregate(all_trips_cleaned$ride_length ~ all_trips_cleaned$member_casual, FUN = max)
aggregate(all_trips_cleaned$ride_length ~ all_trips_cleaned$member_casual, FUN = min)

all_trips_cleaned$member_casual,all_trips_cleaned$ride_length
<chr>,<dbl>
casual,5372.7839
member,795.2523


all_trips_cleaned$member_casual,all_trips_cleaned$ride_length
<chr>,<dbl>
casual,1393
member,508


all_trips_cleaned$member_casual,all_trips_cleaned$ride_length
<chr>,<dbl>
casual,10632022
member,6096428


all_trips_cleaned$member_casual,all_trips_cleaned$ride_length
<chr>,<dbl>
casual,2
member,1


See the average *ride_length* each day for members vs casual cyclists.

In [64]:
all_trips_cleaned$day_of_week <- ordered(all_trips_cleaned$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))
aggregate(all_trips_cleaned$ride_length ~ all_trips_cleaned$member_casual + all_trips_cleaned$day_of_week, FUN = mean)

all_trips_cleaned$member_casual,all_trips_cleaned$day_of_week,all_trips_cleaned$ride_length
<chr>,<ord>,<dbl>
casual,Sunday,5061.3044
member,Sunday,972.9383
casual,Monday,4752.0504
member,Monday,822.3112
casual,Tuesday,4561.8039
member,Tuesday,769.4416
casual,Wednesday,4480.3724
member,Wednesday,711.9838
casual,Thursday,8451.6669


Get the *number_of_rides* and average *ride_length* for every category in *member_casual* and per *weekday*.

In [68]:
summary_alltrips <- all_trips_cleaned %>% 
    mutate(weekday = wday(started_at, label = TRUE)) %>%    # creates weekday field using wday()
    group_by(member_casual, weekday) %>%                    # groups by usertype and weekday
            summarise(number_of_rides = n(),                # calculates the number of rides and average duration 
            average_duration = mean(ride_length)) %>% 		# calculates the average duration
    arrange(member_casual, weekday)							# sorts the records

head(summary_alltrips)

`summarise()` has grouped output by 'member_casual'. You can override using the
`.groups` argument.


member_casual,weekday,number_of_rides,average_duration
<chr>,<ord>,<int>,<dbl>
casual,Sun,18652,5061.304
casual,Mon,5591,4752.050
casual,Tue,7311,4561.804
casual,Wed,7690,4480.372
casual,Thu,7147,8451.667
casual,Fri,8013,6090.737


Write the data into a csv file. The csv file will be used for visualization in Tableau.

In [69]:
write.csv(summary_alltrips, file = 'summary_all_trips.csv')

### Data Visualization and Analysis:

For data visualization, Tableau was used. Data visualization for this case study may be accessed in [my tableau profile](https://public.tableau.com/views/CyclisticPortfolio/Dashboard1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link) or shown in the image below.

![](./cyclistic_data_viz.png)